<a href="https://colab.research.google.com/github/Luthfiyah09/folium_project_n3ts/blob/main/Lutfiyah_Nur_Khansa_X6_Kota_Cirebon.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Import Library

import folium
import pandas as pd
import geopandas as gpd
from google.colab import drive

In [2]:
# Read Data Shapefile

drive.mount('/content/drive')

# Shapefile Admin Indonesia (tidak usah diubah2) Warning! File besar 1.4gb
indonesia_admin = gpd.read_file('/content/drive/MyDrive/ADMIN_INDO/ADMINISTRASI_AR_DESAKEL_10K.shp')

# Pilih Kota/Kabupaten yang ingin ditampilkan, contoh 'Kota Tangerang Selatan'
aoi_admin = indonesia_admin[indonesia_admin['WADMKK'] == 'Kota Cirebon']

Mounted at /content/drive


In [3]:
# CSV file path
csv_path = '/content/drive/MyDrive/Data_Peta/Jumlah_Penduduk.csv'

# Read shapefile
admin_gdf = aoi_admin

# Read CSV data
csv_data = pd.read_csv(csv_path, header = 2, delimiter= ';')

print(csv_data.head())
print(admin_gdf.head())
# Convert column names to lowercase
csv_data['Kelurahan'] = csv_data['Kelurahan'].str.lower().str.replace(' ', '')
admin_gdf['NAMOBJ'] = admin_gdf['NAMOBJ'].str.lower().str.replace(' ', '')
#csv_data['Jumlah_KK'] = csv_data['Jumlah_KK'].astype(str).str.replace('.', '')
#csv_data['Jumlah_KK'] = pd.to_numeric(csv_data['Jumlah_KK'])

# Fix typo to merge properly
#admin_gdf['NAMOBJ'] = admin_gdf['NAMOBJ'].str.replace('pondokjagungt', 'pondokjagung')
#csv_data['Kelurahan'] = csv_data['Kelurahan'].str.replace('perigibaru', 'parigibaru')

# Merge data
merged_data = pd.merge(admin_gdf, csv_data, left_on='NAMOBJ', right_on='Kelurahan', how='left')

# Convert to GeoDataFrame
merged_gdf = gpd.GeoDataFrame(merged_data, geometry=merged_data.geometry)

print(merged_gdf.head())
# Now you can use merged_gdf for further analysis and visualization

    Kelurahan  Jumlah Penduduk  Unnamed: 2  Unnamed: 3  Unnamed: 4  \
0   Argasunya          18717.0         NaN         NaN         NaN   
1    Kalijaga          32115.0         NaN         NaN         NaN   
2  Harjamukti          19818.0         NaN         NaN         NaN   
3      Kecapi          23470.0         NaN         NaN         NaN   
4    Larangan          14885.0         NaN         NaN         NaN   

   Unnamed: 5  Unnamed: 6  Unnamed: 7  
0         NaN         NaN         NaN  
1         NaN         NaN         NaN  
2         NaN         NaN         NaN  
3         NaN         NaN         NaN  
4         NaN         NaN         NaN  
       OBJECTID      NAMOBJ       FCODE REMARK  \
20250   20251.0   Argasunya  BA03070040   None   
20365   20366.0      Drajat  BA03070040   None   
20418   20419.0  Harjamukti  BA03070040   None   
20429   20430.0   Jagasatru  BA03070040   None   
20459   20460.0    Kalijaga  BA03070040   None   

                                      

/usr/local/lib/python3.10/dist-packages/geopandas/geodataframe.py:1819: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


In [5]:
# Create a base map
m = folium.Map(location=[-6.7, 108.55], zoom_start=13)

# Create a choropleth map
choropleth = folium.Choropleth(
    geo_data=merged_gdf,
    name='choropleth',
    data=merged_gdf,
    columns=["Kelurahan", "Jumlah Penduduk"],
    key_on= "feature.properties.NAMOBJ",
    fill_color='YlOrRd',
    fill_opacity=0.7,
    line_opacity=1,
    legend_name="Jumlah Penduduk"
).add_to(m)

jumlah_KK = folium.GeoJson(
    data=merged_gdf,
    name='Jumlah_KK',
    style_function=lambda feature: {'fillOpacity': 0, 'color': 'transparent'},
    popup = folium.GeoJsonPopup(
        fields=['WADMKD','Jumlah Penduduk'],
        aliases=['Kelurahan','Jumlah Penduduk']
    )
).add_to(choropleth)

folium.LayerControl().add_to(m)


m

In [6]:
# Create a base map
m = folium.Map(location=[-6.7, 108.55], zoom_start=13)

# Create a choropleth map
choropleth = folium.Choropleth(
    geo_data=merged_gdf,
    name="Jumlah Penduduk Kota Cirebon 2018	",
    data=merged_gdf,
    columns=["Kelurahan", "Jumlah Penduduk"],
    key_on= "feature.properties.NAMOBJ",
    fill_color='YlOrRd',
    fill_opacity=0.7,
    line_opacity=1,
    legend_name="Jumlah Penduduk"
).add_to(m)

# Create/import data sma
csv_path = '/content/drive/MyDrive/Data_Peta/Data_Sekolah.csv'
data_peta = pd.read_csv(csv_path)

data_peta['geometry'] = gpd.GeoSeries.from_wkt(data_peta['WKT'])

# Extract latitude and longitude from Point geometries
data_peta['latitude'] = data_peta['geometry'].apply(lambda x: x.y)
data_peta['longitude'] = data_peta['geometry'].apply(lambda x: x.x)

# Add markers to the map
# Create a marker layer
marker_layer = folium.FeatureGroup(name="SMAN Cirebon")

# Add markers to the layer
for i in range(0, len(data_peta)):
    folium.Marker([data_peta.iloc[i]['latitude'], data_peta.iloc[i]['longitude']], popup=data_peta.iloc[i]['name']).add_to(marker_layer)

m.add_child(marker_layer)

# Add Jumlah_KK layer map
jumlah_KK = folium.GeoJson(
    data=merged_gdf,
    name='Jumlah_KK',
    style_function=lambda feature: {'fillOpacity': 0, 'color': 'transparent'},
    popup = folium.GeoJsonPopup(
        fields=['NAMOBJ','Jumlah Penduduk'],
        aliases=['Kelurahan','Jumlah Penduduk']
    )
).add_to(choropleth)

folium.LayerControl().add_to(m)


m